In [ ]:
"""
move_file_in_s3

Note: the destination 'folder/directory' needs to exist before you move it.

Required input parameters:
{
  "name_of_target_file": "NAME_OF_YOUR_FILE.txt",
  "target_directory": "test_folder/",
  "s3_bucket_name": "xxx"
}


This is a function that 'moves' a file in AWS-s3 from one place
to another place. More specifically. Before the file is processed it
is in one folder (e.g. a folder for files that have NOT been processed), 
after the file is processed it should be in another folder
(e.g. a folder for files that HAVE been processed).

In AWS-s3, as in a low-level system, the higher-level process of 'moving' 
the file is made up of a series of copy and remove steps:

Step 1. make a copy of the file in the new folder
Step 2. delete the old copy of the file in the original folder

The move_file_in_s3 process can be coded as a 'helper function'
(outside of the main lambda function), with the following input parameters:



The use of try-except is recommended, to handle exceptions etc.

"""

# import python libraries and packages
import boto3
import json

 
# helper function to move files within AWS s3
def move_file_in_s3(s3_resource, s3_bucket_name, target_directory, name_of_target_file, moved_file_destination_in_s3):
  
    # combine to get your full pathway to file
    target_file_and_directory = f"{target_directory}{name_of_target_file}"

    # set your variables
    original_key = target_file_and_directory
    destination_key = moved_file_destination_in_s3 + name_of_target_file

    original_file_data = {
        'Bucket': s3_bucket_name,
        'Key': original_key
    }

    # COPY the file (so then in two places, two copies)
    s3_resource.meta.client.copy(original_file_data, s3_bucket_name, destination_key)

    # Delete the former item (leaving only the moved item)
    s3_resource.Object(s3_bucket_name, original_key).delete()

    return None


"""
# Sample Input: 

{
  "name_of_target_file": "NAME_OF_YOUR_FILE.txt",
  "target_directory": "test_folder/",
  "s3_bucket_name": "xxx"
}
"""
################
# Main Function
################
def lambda_handler(event, context):
 
    #############################################
    # Setup, inputs, connections, variables, etc.
    #############################################

    #####################
    # get moved_file_destination_in_s3
    #####################
    # Test for input:
    try:
        moved_file_destination_in_s3 = event["moved_file_destination_in_s3"]

    except Exception as e:
 
        # if no input specified, set to default:
        moved_file_destination_in_s3 = "files_processed_all_done/"

    #####################
    # get s3_bucket_name
    #####################
    # Test for input:
    try:
        s3_bucket_name = event["s3_bucket_name"]

    except Exception as e:
 
        output = f"""Error: No input for (field=) s3_bucket_name 
        Error Message = '{str(e)} 
        """
        
        # print for terminal
        print(output)

        statusCode = 403

        # End the lambda function
        return {
            'statusCode': statusCode,
            'body': output
        }

    ################################
    # get name_of_target_file in s3
    ################################
    # Test for input:
    try:
        name_of_target_file = event["name_of_target_file"]

    except Exception as e:
 
        output = f"""Error: No input for (field=)name_of_target_file 
        Error Message = '{str(e)} 
        """
        
        # print for terminal
        print(output)

        statusCode = 403

        # End the lambda function
        return {
            'statusCode': statusCode,
            'body': output
        }


    #############################
    # get target_directory in s3
    #############################
    # Test for input:
    try:
        target_directory = event["target_directory"]

        # check that target directory is followed by a '/'
        if target_directory[-1] != '/':
              target_directory = target_directory + '/'

    except Exception as e:
 
        output = f"""Error: No input for (field=)directory_name 
        Error Message = '{str(e)} 
        """
        
        # print for terminal
        print(output)

        statusCode = 403

        # End the lambda function
        return {
            'statusCode': statusCode,
            'body': output
        }


    ####################################
    # S3: Connect to S3 (Make resource)
    ####################################

    try:
        # Set Constants
        AWS_REGION = "us-east-1"

        # make s3_resource
        s3_resource = boto3.resource("s3", region_name=AWS_REGION)

        s3_bucket = s3_resource.Bucket(s3_bucket_name)


    except Exception as e:
 
        output = f"""Error: Could not connect to AWS S3.
        Error Message = '{str(e)} 
        """
        
        # print for terminal
        print(output)

        statusCode = 403

        # End the lambda function
        return {
            'statusCode': statusCode,
            'body': output
        }


    ######################
    # Relocate File In S3
    ######################
    """
    If what you are doing with the file is completed, then move the file
    into a 'we_are_finished_with_these_files' directory.
    """

    try:
        # move file
        move_file_in_s3(s3_resource, s3_bucket_name, target_directory, name_of_target_file, moved_file_destination_in_s3)
        pass


    except Exception as e:
      
        output = f"""Error: Error with relocating files inside s3
        Error Message = '{str(e)}
        """
        # print for terminal
        print(output)

        statusCode = 430

        # End the lambda function
        return {
            'statusCode': statusCode,
            'body': output
        }



    #########
    # Finish
    #########

    return {
        'statusCode': 200,
        'body': json.dumps(f"Completed file moved: '{name_of_target_file}'', OK!")
    }


In [ ]:
import boto3
s3 = boto3.resource('s3')
copy_source = {
    'Bucket': 'mybucket',
    'Key': 'mykey'
}
s3.meta.client.copy(copy_source, 'otherbucket', 'otherkey')